## [Quickstart: Build a Data Application with Snowflake Marketplace, Snowflake Notebooks, and Streamlit in Snowflake](https://quickstarts.snowflake.com/guide/data_apps_summit_lab/)

In [ ]:
USE ROLE accountadmin;
USE WAREHOUSE compute_wh;
USE DATABASE weather_lab;
USE SCHEMA weather_schema;

In [ ]:
-- Explore what weather variables are available
SELECT DISTINCT variable_name
FROM WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_METRICS_TIMESERIES
LIMIT 20;

In [ ]:
-- Average temperature by state, in Celsius
SELECT 
    idx.state_name,
    AVG(ts.value) as avg_temperature
FROM WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_METRICS_TIMESERIES ts
JOIN WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_STATION_INDEX idx 
    ON ts.noaa_weather_station_id = idx.noaa_weather_station_id
WHERE ts.variable_name = 'Average Temperature'
    AND ts.date >= '2020-01-01'
    AND idx.country_geo_id = 'country/USA'
GROUP BY idx.state_name
ORDER BY avg_temperature DESC;

In [ ]:
-- Sample temperature data for zip codes for weather stations in California
SELECT 
    ts.date,
    ts.value as temperature,
    idx.noaa_weather_station_name,
    idx.zip_name,
    idx.state_name
FROM WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_METRICS_TIMESERIES ts
JOIN WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_STATION_INDEX idx 
    ON ts.noaa_weather_station_id = idx.noaa_weather_station_id
WHERE ts.variable_name = 'Average Temperature'
    AND idx.state_name = 'Arizona'
    AND ts.date >= '2023-01-01'
    AND idx.zip_name IS NOT NULL
ORDER BY ts.date DESC;

In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.types import IntegerType, FloatType, StringType
from snowflake.snowpark.functions import avg, sum, col, udf, call_udf, call_builtin, year, month, dayofyear
import pandas as pd
from datetime import date
import numpy as np

# scikit-learn for training ML models, used later in notebook
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

session = Session.builder.getOrCreate()

# Snowpark DataFrame equivalent to previous SQL query
ts = session.table("WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_METRICS_TIMESERIES")
idx = session.table("WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_STATION_INDEX")

sample_weather_df = (
    ts.join(idx, ts.col("NOAA_WEATHER_STATION_ID") == idx.col("NOAA_WEATHER_STATION_ID"))
    .filter(ts.col('VARIABLE_NAME') == 'Average Temperature')
    .filter(idx.col('STATE_NAME') == 'Arizona')
    .filter(ts.col('DATE') >= '2023-01-01')
    .filter(idx.col('ZIP_NAME').isNotNull())
    .select(
        ts.col('DATE'),
        ts.col('VALUE').alias('TEMPERATURE'),
        idx.col('NOAA_WEATHER_STATION_NAME'),
        idx.col('ZIP_NAME'),
        idx.col('STATE_NAME')
    )
    .orderBy(ts.col('DATE').desc())
    .limit(100)
)

sample_weather_df.show(20)

In [ ]:
# Create Snowpark DataFrame for temperature data
# Temps in Celsius
print("Creating Snowpark DataFrame...")

ts = session.table("WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_METRICS_TIMESERIES")
idx = session.table("WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_STATION_INDEX")

# Create weather dataframe
weather_df = (
    ts.join(idx, ts.col("NOAA_WEATHER_STATION_ID") == idx.col("NOAA_WEATHER_STATION_ID"))
    .filter(ts.col('VARIABLE_NAME') == 'Average Temperature')
    .filter(idx.col('COUNTRY_GEO_ID') == 'country/USA')
    .filter(ts.col('DATE') >= '2000-01-01')
    .filter(idx.col('STATE_NAME').isNotNull())
    .select(
        ts.col('DATE'),
        ts.col('VALUE').alias('TEMPERATURE'),
        idx.col('STATE_NAME'),
        call_builtin('YEAR', ts.col('DATE')).alias('YEAR'),
        call_builtin('MONTH', ts.col('DATE')).alias('MONTH')
    )
    .orderBy(ts.col('DATE'))
)

print("Sample of weather DataFrame:")
weather_df.limit(10).show()

print(f"Total records in weather DataFrame: {weather_df.count()}")

# Create monthly averages by state for model training
print("Creating features for ML training...")

monthly_temp_df = (
    weather_df
    .groupBy(col('STATE_NAME'), col('YEAR'), col('MONTH'))
    .agg(avg(col('TEMPERATURE')).alias('AVG_TEMP'))
    .orderBy(col('STATE_NAME'), col('YEAR'), col('MONTH'))
)

print("Monthly temperature averages by state:")
monthly_temp_df.limit(10).show()

# Focus on Arizona data for our location-specific models
print("Preparing Arizona data for location-specific training...")

az_weather_pd = (
    monthly_temp_df
    .filter(col('STATE_NAME') == 'Arizona')
    .filter(col('YEAR') >= 2000)
    .select(col('YEAR'), col('MONTH'), col('AVG_TEMP'))
    .orderBy(col('YEAR'), col('MONTH'))
    .to_pandas()
)

print(f"Arizona temperature data shape: {ca_weather_pd.shape}")

print("\nSample data:")
print(az_weather_pd.head())

In [ ]:
# Train model for 90210
print("Training weather model for zip code 86005...")

# Target zip code
target_zips = {
  '86005': 'FLAGSTAFF ARIZONA'
}

# Function to calculate approx. distance from coast
def distance_from_coast(latitude, longitude):
    """
    Approximate distance from Pacific coast to any point in Arizona.
    Uses a simplified model based on longitude and a fixed coastal reference.
    """

    # Use Pacific Coast reference near San Diego
    coast_lon = -117.25

    # Use different miles-per-degree based on latitude (optional for more accuracy)
    if latitude > 35:
        miles_per_deg = 53.3  # Northern AZ
    elif latitude > 33:
        miles_per_deg = 53.0  # Central AZ
    else:
        miles_per_deg = 52.7  # Southern AZ

    # Compute distance
    distance = abs(longitude - coast_lon) * miles_per_deg

    return distance

# Prepare training data for zip code
models = {}
zip_data = {}

for zip_code, description in target_zips.items():
  
  # Retrieve >20 years of data for this specific zip code
  zip_query = f"""
  SELECT 
      ts.date,
      ts.value as temperature,
      idx.latitude,
      idx.longitude, 
      idx.elevation,
      EXTRACT(MONTH FROM ts.date) as month,
      EXTRACT(YEAR FROM ts.date) as year
  FROM WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_METRICS_TIMESERIES ts
  JOIN WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_STATION_INDEX idx 
      ON ts.noaa_weather_station_id = idx.noaa_weather_station_id
  WHERE ts.variable_name = 'Average Temperature'
      AND idx.zip_name = '{zip_code}'
      AND ts.date >= '2000-01-01'
      AND ts.date < '2025-01-01'
      AND ts.value IS NOT NULL
  ORDER BY ts.date
  """
  
  # Get the data
  zip_weather_df = session.sql(zip_query).to_pandas()
  
  if not zip_weather_df.empty:
      print(f"Found {len(zip_weather_df)} records from {zip_weather_df['YEAR'].min()}-{zip_weather_df['YEAR'].max()}")
      
      # Calculate additional geographic features
      if len(zip_weather_df) > 0:
          lat = zip_weather_df['LATITUDE'].iloc[0]
          lon = zip_weather_df['LONGITUDE'].iloc[0]
          elev = zip_weather_df['ELEVATION'].iloc[0]
          
          
          # Calculate distance from coast
          coast_distance = distance_from_coast(lat, lon)
          
          # Prepare features for training
          # Create monthly averages with geographic features
          monthly_data = (
              zip_weather_df.groupby(['YEAR', 'MONTH'])
              .agg({
                  'TEMPERATURE': 'mean',
                  'LATITUDE': 'first',
                  'LONGITUDE': 'first', 
                  'ELEVATION': 'first'
              })
              .reset_index()
          )
          
          # Add calculated features
          monthly_data['COAST_DISTANCE'] = coast_distance
          monthly_data['MONTH_SIN'] = np.sin(2 * np.pi * monthly_data['MONTH'] / 12)
          monthly_data['MONTH_COS'] = np.cos(2 * np.pi * monthly_data['MONTH'] / 12)
          
          # Prepare features (X) and target (y)
          feature_columns = ['MONTH', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'COAST_DISTANCE', 'MONTH_SIN', 'MONTH_COS']
          X = monthly_data[feature_columns].values
          y = monthly_data['TEMPERATURE'].values
          
          print(f"Training features: {feature_columns}")
          print(f"No. of training samples: {len(X)}")
          
          # Train the model
          model = LinearRegression()
          model.fit(X, y)
          
          # Store model and metadata
          models[zip_code] = {
              'model': model,
              'features': feature_columns,
              'latitude': lat,
              'longitude': lon,
              'elevation': elev,
              'coast_distance': coast_distance,
              'training_samples': len(X)
          }
          
          # Store sample data for testing
          zip_data[zip_code] = monthly_data
          
      else:
          print(f"No valid data found for {zip_code}")
  else:
      print(f"No data found for {zip_code}")

print(f"Successfully trained model for {len(models)} zip code")

In [ ]:
def predict_temperature(zip_code: str, month: int) -> float:
    # Get the trained model and geographic data
    model_info = models[zip_code]
    model = model_info['model']
    
    # Create features
    features = [
        month,
        model_info['latitude'],
        model_info['longitude'], 
        model_info['elevation'],
        model_info['coast_distance'],
        np.sin(2 * np.pi * month / 12),  # MONTH_SIN
        np.cos(2 * np.pi * month / 12)   # MONTH_COS
    ]
    
    # Use the trained model
    prediction = model.predict([features])[0]
    return round(prediction, 1)

# Register the UDF
print("Creating UDF...")
predict_temp_udf = session.udf.register(
   predict_temperature,
   return_type=FloatType(),
   input_types=[StringType(), IntegerType()],
   packages=["pandas", "scikit-learn", "numpy"],
   is_permanent=True,
   name="predict_temperature_udf",
   replace=True,
   stage_location="@weather_schema.udf_stage"
)

print("UDF registered successfully")

# Test the UDF 
print("Testing predictions:")
test_cases = [
   ('86005', 7), # Flagstaff in July
   ('86005', 1)  # Flagstaff in Jan
]

month_names = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

print(f"{'Zip Code':<8} {'Month':<12} {'Prediction':<12} {'Location'}")
print("-" * 50)

for zip_code, month in test_cases:
   result = session.sql(f"SELECT predict_temperature_udf('{zip_code}', {month}) as temp").collect()
   temp = result[0]['TEMP']
   location = target_zips.get(zip_code, 'Unknown')
   print(f"{zip_code:<8} {month_names[month]:<12} {temp:.1f}°C        {location}")

print("Weather prediction model is ready!")